In [1]:
import torch
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
animals = ['cavallo', 'pecora', 'gatto', 'farfalla', 'cane']
datasetDir = "Dataset/"

In [3]:
def generateDatasetCSV():
    newlines = []
    for i,animal in enumerate(animals):
        
        with open(f"{datasetDir+animal}.csv","r") as f:
            lines = f.readlines()
            for line in lines:
                newlines.append(','.join(line.strip().split(",")[1:49])+f",{i}")
    
    random.shuffle(newlines)
    train_lines = newlines[0:int(0.8*len(newlines))]
    test_lines = newlines[int(0.8*len(newlines)):len(newlines)]
    
    with open(f"{datasetDir}train.csv","w") as f:
        for line in train_lines:
            f.write(line)
            f.write("\n")
    with open(f"{datasetDir}test.csv","w") as f:
        for line in test_lines:
            f.write(line)
            f.write("\n")
            
    
            
  
            
    

        
                
        
            
    

In [ ]:
generateDatasetCSV()

In [4]:
class AnimalDataset(Dataset):
    def __init__(self,data_source):
        with open(data_source,"r") as f:
            self.lines = f.readlines()
        self.inputs = []
        self.outputs = []
        for line in self.lines[0:]:            
            self.linedata = tuple(map(float,line.strip().split(",")))
            self.inputs.append(torch.tensor(self.linedata[0:48])/189000.0)
            self.outputs.append(torch.tensor(self.linedata[48]))
    def __len__(self):
        return len(self.outputs)
    def __getitem__(self,idx):
        return self.inputs[idx],self.outputs[idx]

In [5]:
class AANN1(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(in_features=48,out_features=64),nn.ReLU(),nn.Linear(in_features=64,out_features=24))
        self.decoder = nn.Sequential(nn.Linear(in_features=24,out_features=64),nn.ReLU(),nn.Linear(in_features=64,out_features=48))
    def forward(self,x,path="all"):
        if(path == "all"):
            return self.decoder(self.encoder(x))
        else:
            return self.encoder(x)
    

In [6]:
class AANN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(in_features=24,out_features=32),nn.ReLU(),nn.Linear(in_features=32,out_features=12))
        self.decoder = nn.Sequential(nn.Linear(in_features=12,out_features=32),nn.ReLU(),nn.Linear(in_features=32,out_features=24))
    def forward(self,x,path="all"):
        if(path == "all"):
            return self.decoder(self.encoder(x))
        else:
            return self.encoder(x)

In [7]:
class AANN3(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(in_features=12,out_features=18),nn.ReLU(),nn.Linear(in_features=18,out_features=6))
        self.decoder = nn.Sequential(nn.Linear(in_features=6,out_features=18),nn.ReLU(),nn.Linear(in_features=18,out_features=12))
    def forward(self,x,path="all"):
        if(path == "all"):
            return self.decoder(self.encoder(x))
        else:
            return self.encoder(x)

In [8]:
trainDataset = AnimalDataset(data_source=f"{datasetDir}train.csv")


In [9]:
batch_size = 25
trainLoader  = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size,shuffle=True)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
aann1 = AANN1().to(device)
aann2 = AANN2().to(device)
aann3 = AANN3().to(device)



cuda:0


In [11]:
# Training AANN1

aann1_optimizer = optim.Adam(aann1.parameters(),lr=4e-4)
criterion = nn.KLDivLoss(reduction="batchmean")
epochs = 1000

for epoch in range(epochs):
    total_loss = 0
    cnt = 0
    for i,(data,target) in enumerate(trainLoader):
        data = data.to(device)
        
        out = aann1(data)
        
        loss = criterion(nn.functional.log_softmax(out),data)
        cnt+=1
        total_loss += loss.item() 
        aann1_optimizer.zero_grad()
        loss.backward()

        aann1_optimizer.step()
    
    print(f"Epoch {epoch} average loss: {total_loss/cnt}")




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Epoch 0 average loss: 0.366139590242313
Epoch 1 average loss: 0.1792977399643964
Epoch 2 average loss: 0.15218448007661572
Epoch 3 average loss: 0.14527538315635308
Epoch 4 average loss: 0.14041318403010153
Epoch 5 average loss: 0.1365538525055018
Epoch 6 average loss: 0.13268991902932917
Epoch 7 average loss: 0.12738785339710537
Epoch 8 average loss: 0.12085021438348265
Epoch 9 average loss: 0.11556413038889378
Epoch 10 average loss: 0.11133268144790767
Epoch 11 average loss: 0.10663148973288286
Epoch 12 average loss: 0.09981274948875489
Epoch 13 average loss: 0.09352556328002479
Epoch 14 average loss: 0.08995512613319838
Epoch 15 average loss: 0.0870273409673593
Epoch 16 average loss: 0.084114116120879
Epoch 17 average loss: 0.08182541239197613
Epoch 18 average loss: 0.08019176210879142
Epoch 19 average loss: 0.07877888851454548
Epoch 20 average loss: 0.07735646197790177
Epoch 21 average loss: 0.07583924390926851
Epoch 22 average loss: 0.07413284331204499
Epoch 23 average loss: 0.072

In [12]:
torch.save(aann1.state_dict(), "aann1.pth")

In [ ]:
trainDataset[0][0]*189000.0

tensor([ 4609.0000,  9129.0000, 12030.0000, 10840.0000,  5547.0000,  3127.0000,
         2927.0000,  2348.0000,  1321.0000,   735.0000,   413.0000,   238.0000,
          108.0000,   104.0000,   189.0000,  9335.0000,  1090.0000,  1513.0000,
         1950.0000,  2477.0000,  4344.0000,  6675.0005,  8671.0000, 10078.0000,
         8745.0000,  4796.0000,  2227.0000,   626.0000,   149.0000,   105.0000,
          867.9999,  8686.0000,   731.0000,  1417.0000,  2396.0000,  4185.0000,
         7035.9995,  9574.0000, 10475.0000,  7965.0000,  5578.0000,  2620.0000,
          930.0000,   344.0000,   177.0000,   292.0000,  4079.0000,  5201.0000])

In [22]:
torch.nn.functional.softmax(aann1(trainDataset[0][0].to(device)))*189000.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([ 4747.7466,  8092.1455, 12898.1094, 10660.0244,  5550.7769,  3304.1599,
         2719.3201,  2255.3398,  1436.8962,   748.5262,   334.3553,   175.7560,
          129.3499,   113.9629,   339.8289,  9307.3447,   860.8162,  1440.6243,
         1879.3680,  2696.9561,  4158.5957,  6708.2373,  8794.8984, 10656.1514,
         8597.2461,  4779.0674,  1854.5524,   651.5597,   278.5815,   170.8670,
          567.5543,  8678.9893,   878.5538,  1525.9169,  2182.2974,  4385.0981,
         7804.2456,  9586.4434,  9236.3516,  7734.3130,  4679.4995,  2445.9309,
         1034.6328,   384.5521,   325.9367,   892.0481,  3284.9761,  7031.4829],
       device='cuda:0', grad_fn=<MulBackward0>)

In [25]:
# Training AANN2

aann2_optimizer = optim.Adam(aann2.parameters(),lr=4e-4)
criterion = nn.MSELoss()
epochs = 1000

for epoch in range(epochs):
    total_loss = 0
    cnt = 0
    for i,(data,target) in enumerate(trainLoader):
        data = aann1(data.to(device),path="encoder")
        
        out = aann2(data)
        
        loss = criterion(out,data)
        cnt+=1
        total_loss += loss.item() 
        aann2_optimizer.zero_grad()
        loss.backward()

        aann2_optimizer.step()
    
    print(f"Epoch {epoch} average loss: {total_loss/cnt}")


Epoch 0 average loss: 0.05817492023946135
Epoch 1 average loss: 0.0234093066940622
Epoch 2 average loss: 0.016917811002057748
Epoch 3 average loss: 0.012536243079919235
Epoch 4 average loss: 0.010676306614603377
Epoch 5 average loss: 0.009634478415555914
Epoch 6 average loss: 0.009011010683824027
Epoch 7 average loss: 0.008612732891980248
Epoch 8 average loss: 0.008357800321591249
Epoch 9 average loss: 0.00818156004052049
Epoch 10 average loss: 0.008041850903553796
Epoch 11 average loss: 0.00791219105890692
Epoch 12 average loss: 0.007783954460196072
Epoch 13 average loss: 0.007669766896419885
Epoch 14 average loss: 0.0075574789239596
Epoch 15 average loss: 0.007466273750167402
Epoch 16 average loss: 0.007358407447234614
Epoch 17 average loss: 0.007265234169903653
Epoch 18 average loss: 0.0071561822133152115
Epoch 19 average loss: 0.007076006777237807
Epoch 20 average loss: 0.006995551817136903
Epoch 21 average loss: 0.006937091523861437
Epoch 22 average loss: 0.006883364375836276
Epoc

In [26]:
torch.save(aann2.state_dict(), "aann2.pth")

In [27]:
aann1(trainDataset[0][0].to(device),path="encoder")

tensor([ 0.3304, -0.0725,  0.1704,  0.6812, -0.1799,  0.1973,  0.0968,  0.7235,
        -0.1714,  0.1652, -0.2332, -0.1682,  0.5249, -0.2326, -0.4005,  0.1293,
        -0.1510,  0.1715, -0.2592,  0.0832,  0.3252,  0.1327, -0.2641,  0.1402],
       device='cuda:0', grad_fn=<AddBackward0>)

In [28]:
aann2(aann1(trainDataset[0][0].to(device),path="encoder"))

tensor([ 0.3999, -0.0325,  0.1613,  0.6534, -0.1389,  0.1584,  0.1286,  0.7048,
        -0.1274,  0.1852, -0.2267, -0.1324,  0.5030, -0.2572, -0.3672,  0.1172,
        -0.0631,  0.2647, -0.2267,  0.0094,  0.3383,  0.1653, -0.2778,  0.0988],
       device='cuda:0', grad_fn=<AddBackward0>)

In [29]:
# Training AANN3

aann3_optimizer = optim.Adam(aann3.parameters(),lr=4e-4)
criterion = nn.MSELoss()
epochs = 1000

for epoch in range(epochs):
    total_loss = 0
    cnt = 0
    for i,(data,target) in enumerate(trainLoader):
        data = aann2(aann1(data.to(device),path="encoder"),path="encoder")
        
        out = aann3(data)
        
        loss = criterion(out,data)
        cnt+=1
        total_loss += loss.item() 
        aann3_optimizer.zero_grad()
        loss.backward()

        aann3_optimizer.step()
    
    print(f"Epoch {epoch} average loss: {total_loss/cnt}")


Epoch 0 average loss: 0.15512345195136662
Epoch 1 average loss: 0.07137461564101297
Epoch 2 average loss: 0.045075845457880195
Epoch 3 average loss: 0.03223217687256154
Epoch 4 average loss: 0.028633010366971568
Epoch 5 average loss: 0.02710459353335575
Epoch 6 average loss: 0.026296621484144064
Epoch 7 average loss: 0.025802247141774627
Epoch 8 average loss: 0.02537340465221812
Epoch 9 average loss: 0.024974416185677195
Epoch 10 average loss: 0.02464267824440839
Epoch 11 average loss: 0.02428609356817596
Epoch 12 average loss: 0.023935885121934737
Epoch 13 average loss: 0.023541845354587195
Epoch 14 average loss: 0.023142966941923686
Epoch 15 average loss: 0.022774282491683676
Epoch 16 average loss: 0.022449091014614536
Epoch 17 average loss: 0.022185235148682515
Epoch 18 average loss: 0.02191641017892409
Epoch 19 average loss: 0.021690079613382037
Epoch 20 average loss: 0.02146965656251711
Epoch 21 average loss: 0.02126864598383838
Epoch 22 average loss: 0.021050480276630715
Epoch 23

In [30]:
torch.save(aann3.state_dict(), "aann3.pth")